In [1]:
import geopandas as gpd
import math
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
poi_df = pd.read_csv('./exports/poi.csv')
poi_df.head()

,placekey,cbg,naics_code,category,sub_category,location_name,area_square_feet,latitude,longitude
0,226-222@627-s4n-pqf,360610158022,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Unna Bakery,NaN,40.787027,-73.950435
1,225-225@627-s99-9xq,360810849002,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Fay Da Bakery,24759.0,40.756244,-73.829009
2,225-225@627-vsw-7nq,360850146071,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Amici Bakery,16166.0,40.559894,-74.166016
3,22b-222@627-s7s-jqf,360810475001,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Sunrise Bakery & Cafe,732.0,40.736802,-73.879306
4,23h-222@627-s9z-n3q,360810779071,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Avraham Bakery,12069.0,40.724409,-73.820241


In [3]:
FAST_FOOD_NAMES = set([
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4783380/
    "McDonald's",
    "Burger King",
    "Starbucks",
    "Dunkin'",
    "Pizza Hut",
    "Subway",
    "Taco Bell",
    "KFC",
    "Chick-fil-A",
    "Wendy's",
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2446463/
    "Au Bon Pain",
    "Papa John's",
    "Popeyes Louisiana Kitchen",
# https://dol.ny.gov/system/files/documents/2021/07/p716.pdf
    "Ben & Jerry's",
    "Chipotle Mexican Grill",
    "Golden Krust Caribbean Bakery and Grill",
    "Jamba",
    "Nathan's Famous",
    "Shake Shack",
    "Tim Hortons",
    "Uno Chicago Grill",
    "White Castle",
# https://s27147.pcdn.co/wp-content/uploads/NELP-Fact-Sheet-Fast-Food-Employment-New-York.pdf
    "Baskin Robbins",
    "Carvel",
    "Little Caesers Pizza",
    "Domino's Pizza",
    "Panera Bread",
    "Auntie Anne's",
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4967005/
    "Arby's",
])

In [4]:
poi_df.loc[poi_df['location_name'].isin(FAST_FOOD_NAMES), 'category'] = 'Fast-Food Restaurants'
poi_df.loc[poi_df['location_name'].isin(FAST_FOOD_NAMES), :].head()

,placekey,cbg,naics_code,category,sub_category,location_name,area_square_feet,latitude,longitude
14743,232-222@627-wg6-vs5,360470968001,722511,Fast-Food Restaurants,Full-Service Restaurants,Golden Krust Caribbean Bakery and Grill,605.0,40.645151,-73.902657
14790,234-223@627-s6k-6rk,360050151003,722511,Fast-Food Restaurants,Full-Service Restaurants,Golden Krust Caribbean Bakery and Grill,10769.0,40.831446,-73.900305
15467,234-222@627-s26-zfz,360810632002,722511,Fast-Food Restaurants,Full-Service Restaurants,Golden Krust Caribbean Bakery and Grill,2747.0,40.675867,-73.740355
15610,227-222@627-wff-99f,360470058002,722511,Fast-Food Restaurants,Full-Service Restaurants,Uno Chicago Grill,8230.0,40.618012,-74.029958
15935,22x-222@627-s8z-j7q,360470329004,722511,Fast-Food Restaurants,Full-Service Restaurants,Golden Krust Caribbean Bakery and Grill,875.0,40.663697,-73.951163


In [5]:
poi_df.head()

,placekey,cbg,naics_code,category,sub_category,location_name,area_square_feet,latitude,longitude
0,226-222@627-s4n-pqf,360610158022,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Unna Bakery,NaN,40.787027,-73.950435
1,225-225@627-s99-9xq,360810849002,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Fay Da Bakery,24759.0,40.756244,-73.829009
2,225-225@627-vsw-7nq,360850146071,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Amici Bakery,16166.0,40.559894,-74.166016
3,22b-222@627-s7s-jqf,360810475001,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Sunrise Bakery & Cafe,732.0,40.736802,-73.879306
4,23h-222@627-s9z-n3q,360810779071,311811,Supermarkets and Specialty Food Stores,Retail Bakeries,Avraham Bakery,12069.0,40.724409,-73.820241


In [6]:
poi_df.to_csv('./exports/poi_health_recategorized.csv', index=None)